### ==========================================================================================
# PART 3: COVNET MODEL EVOLUTIONARY TRAINING
### ==========================================================================================

In [1]:
import os
import re
import random
import pandas as pd
import tensorflow as tf
from datetime import datetime
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
# project folder
path_project = 'E:/DeepLearning/breast_cancer/IDC_regular_ps50_idx5/'

# set train, validate, and test folders
dir_train = os.path.join(path_project, '_train')
dir_valid = os.path.join(path_project, '_validate')

### Training Functions

In [3]:
def train_model(model_x, model_id, lr, epochs, epoch_steps, valid_steps, batch_size, image_size):
    model = model_x(lr=lr, image_size=image_size)
    
    print('Model:', model_x)
    print('Batch Size:', batch_size)
    print('Learning Rate:', lr)
    print('Epochs:', epochs)
    print('Steps per Epoch:', epoch_steps)
    print('Validation Steps:', valid_steps)
    print('Image Size:', image_size)
    print('_' * 65, '\n')
        
    print('Training Set:')    
    datagen_train = ImageDataGenerator(rescale=1./255)
    gen_train = datagen_train.flow_from_directory(dir_train, target_size=image_size, batch_size=batch_size, class_mode='binary')
    
    print('Validation Set:')    
    datagen_valid = ImageDataGenerator(rescale=1./255)
    gen_valid = datagen_valid.flow_from_directory(dir_valid, target_size=image_size, batch_size=batch_size, class_mode='binary')
    print('_' * 65, '\n')

    print('Model training started...\n')
    
    t0 = datetime.now()
    with tf.device('/gpu:0'):
        history = model.fit_generator(gen_train, steps_per_epoch=epoch_steps, epochs=epochs, validation_data=gen_valid, 
                                                                                                validation_steps=valid_steps)
    
    elpased_time = str(datetime.now() - t0).split('.')[0]    
    hours = elpased_time.split(':')[0]
    minutes = elpased_time.split(':')[1]
    seconds = elpased_time.split(':')[2]
    
    print('\nTraining process completed in:', int(hours), 'h', int(minutes), 'm', int(seconds), 's')            
    
    # save model
    models_folder = 'MODELS'    
    if not os.path.exists(os.path.join(path_project, models_folder)):
        os.makedirs(os.path.join(path_project, models_folder))
    filename = f"model_breast_cancer_{model_id}.h5"
    model.save(os.path.join(path_project, models_folder, filename))
    print('Model saved as: "', models_folder, '/', filename, '"')
    
    print('#' * 120, '\n')
    
    return (history, elpased_time, model)


def train_generation(model, image_size, gen):
    # read generations file
    df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')    

    for row in df_generations.itertuples():
        # process only untrained rows
        if row.train_loss == '' and row.val_acc == '':
            try:
                print('Processing Model', row.Index + 1, 'of', len(df_generations))
                logs = train_model(model, 
                                   model_id = row.Index,
                                   lr = row.learning_rate, 
                                   epochs = row.epochs, 
                                   epoch_steps = row.steps_per_epoch, 
                                   valid_steps = row.validation_steps, 
                                   batch_size = row.batch_size, 
                                   image_size = image_size)

                df_generations.loc[row.Index, 'train_loss'] = logs[0].history['loss'][-1]
                df_generations.loc[row.Index, 'train_acc'] = logs[0].history['acc'][-1]
                df_generations.loc[row.Index, 'val_loss'] = logs[0].history['val_loss'][-1]
                df_generations.loc[row.Index, 'val_acc'] = logs[0].history['val_acc'][-1]
                df_generations.loc[row.Index, 'time'] = logs[1]
                df_generations.loc[row.Index, 'gen'] = gen

                # save results to csv after each iteration
                df_generations.to_csv(os.path.join(path_project, 'generations.csv'), index=None)    
            except:
                pass
    
    return df_generations             

### Define Model Architecture

In [4]:
def model_7(lr, image_size):
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr), metrics=['acc'])        
    model.summary()    
    return model

### Define Genetic Algorithms

In [9]:
colnames = ['batch_size', 'steps_per_epoch', 'validation_steps', 'epochs', 'learning_rate',  
                                                    'train_loss', 'train_acc', 'val_loss', 'val_acc', 'time', 'gen', 'alive']

def first_generation(size):    
    df_out = pd.DataFrame()
    
    for i in range(size):
    
        # read generations file if it exists, or create new if it doesn't
        if os.path.exists(os.path.join(path_project, 'generations.csv')):
            df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv'))
        else:
            df_generations = pd.DataFrame(columns=colnames)

        # create individual
        df_new = pd.DataFrame([[random.choice(range_batch_size),
                                random.choice(range_epoch_steps),
                                random.choice(range_valid_steps),
                                random.choice(range_epochs),
                                1/10**random.choice(range_lr),
                                '', '', '', '', '', '', '', '', '1', True]], columns=colnames)    
        df_temp = df_generations.append(df_new)

        # regenerate individual if already exists in df_generations
        while len(df_temp[df_temp.duplicated(keep=False)]) > 0:  
            df_generations = df_generations.drop_duplicates()
            df_new = pd.DataFrame([[random.choice(range_batch_size),
                                    random.choice(range_epoch_steps),
                                    random.choice(range_valid_steps),
                                    random.choice(range_epochs),
                                    1/10**random.choice(range_lr),
                                    '', '', '', '', '', '', '', '', '1', True]], columns=colnames)        
            df_temp = df_generations.append(df_new)

        df_out = df_out.append([df_new]).reset_index(drop=True)
            
        # save new parent to generations file
        df_temp.to_csv(os.path.join(path_project, 'generations.csv'), index=None)
            
    return df_out


def select_fittest(top_percent):
    # read generations file
    df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv'))
    df_generations = df_generations.fillna('')

    # select a pool of only alive individuals
    df_alive = df_generations[df_generations.alive]

    # extinguish non-fit algorithms based on validation accuracy
    non_survivors = df_alive.nsmallest(round(len(df_alive) * (1 - top_percent / 100)), 'val_acc').index.tolist()    
    df_generations.loc[non_survivors, 'alive'] = False

    # save file
    df_generations.to_csv(os.path.join(path_project, 'generations.csv'), index=None)
    
    return df_generations[df_generations.alive]


def create_parents():    
    # read generations file
    df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv'))
    
    # select a pool of only alive individuals
    df_pool = df_generations[df_generations.alive]
    
    # select parents at random from pool for as long as pairs can be matched 
    matched_parents = []
    while len(df_pool) > 1:    
        df_parents = df_pool.sample(2)
        new_parents = tuple(df_parents.index)
        matched_parents.append(new_parents)

        # remove new parents from pool        
        df_pool = df_pool.drop([new_parents[0], new_parents[1]], axis=0)

    return matched_parents


def create_child(df_parents, generation, mutate=False):
    df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv'))
            
    mutate_choice = random.choice(['batch_size', 'steps_epoch', 'valid_steps', 'epochs', 'learning_rate'])  

    if mutate and mutate_choice == 'batch_size':
        batch_size = random.choice(range_batch_size)
    else:
        batch_size = int(df_parents.batch_size.sample())

    if mutate and mutate_choice == 'steps_epoch':
        steps_per_epoch = random.choice(range_epoch_steps)
    else:
        steps_per_epoch = int(df_parents.steps_per_epoch.sample())

    if mutate and mutate_choice == 'valid_steps':
        validation_steps = random.choice(range_valid_steps)
    else:
        validation_steps = int(df_parents.validation_steps.sample())

    if mutate and mutate_choice == 'epochs':
        epochs = random.choice(range_epochs)
    else:
        epochs = int(df_parents.epochs.sample())

    if mutate and mutate_choice == 'learning_rate':
        learning_rate = 1/10**random.choice(range_lr)
    else:
        learning_rate = float(df_parents.learning_rate.sample())            
        
    df_child = pd.DataFrame([[batch_size, steps_per_epoch, validation_steps, epochs, learning_rate,
                                                      '', '', '', '', '', generation, True]], columns=colnames)    
    df_temp = df_generations.append(df_child)
    
    # regenerate child if already exists in previous generations
    while len(df_temp[df_temp.duplicated(keep=False)]) > 0:  
        df_generations = df_generations.drop_duplicates()
        df_child = pd.DataFrame([[batch_size, steps_per_epoch, validation_steps, epochs, learning_rate,
                                      '', '', '', '', '', generation, True]], columns=colnames)    
        df_temp = df_generations.append(df_child)
        
    # save new child to generations file
    df_temp.to_csv(os.path.join(path_project, 'generations.csv'), index=None)
        
    return df_child

### Set Up Hyperparameter Combinations

In [6]:
range_epochs = range(10, 101, 10)
range_epoch_steps = range(10, 201, 10)
range_valid_steps = range(10, 101, 10)
range_batch_size = range(20, 201, 20)
range_lr = range(1, 6)

## 1st GENERATION ------------------------------------------------------------------

In [7]:
# Create first generation
first_generation(1000).tail()

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
995,200,170,90,50,0.001,,,,,,1,True
996,160,140,10,80,0.001,,,,,,1,True
997,160,170,100,80,0.100,,,,,,1,True
998,180,190,100,70,0.001,,,,,,1,True
999,140,40,10,50,0.001,,,,,,1,True


In [8]:
# Train and validate
train_generation(model=model_7, image_size=(50,50), gen=1)

Processing Model 968 of 1000
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 128) 

130/130 [==============================] - 10s 76ms/step - loss: 0.3239 - acc: 0.8626 - val_loss: 0.3215 - val_acc: 0.8685
Epoch 75/100
130/130 [==============================] - 10s 77ms/step - loss: 0.3256 - acc: 0.8633 - val_loss: 0.3174 - val_acc: 0.8628
Epoch 76/100
130/130 [==============================] - 10s 78ms/step - loss: 0.3240 - acc: 0.8618 - val_loss: 0.3163 - val_acc: 0.8672
Epoch 77/100
130/130 [==============================] - 10s 79ms/step - loss: 0.3282 - acc: 0.8590 - val_loss: 0.3389 - val_acc: 0.8560
Epoch 78/100
130/130 [==============================] - 10s 75ms/step - loss: 0.3227 - acc: 0.8628 - val_loss: 0.3021 - val_acc: 0.8678
Epoch 79/100
130/130 [==============================] - 10s 77ms/step - loss: 0.3170 - acc: 0.8674 - val_loss: 0.3196 - val_acc: 0.8633
Epoch 80/100
130/130 [==============================] - 10s 78ms/step - loss: 0.3176 - acc: 0.8655 - val_loss: 0.3305 - val_acc: 0.8588
Epoch 81/100
130/130 [==============================] - 10s 7

Found 222020 images belonging to 2 classes.
Validation Set:
Found 27752 images belonging to 2 classes.
_________________________________________________________________ 

Model training started...

Epoch 1/70
160/160 [==============================] - 11s 70ms/step - loss: 0.6191 - acc: 0.7075 - val_loss: 0.5994 - val_acc: 0.7141
Epoch 2/70
160/160 [==============================] - 11s 67ms/step - loss: 0.6004 - acc: 0.7129 - val_loss: 0.5878 - val_acc: 0.7254
Epoch 3/70
160/160 [==============================] - 11s 67ms/step - loss: 0.5948 - acc: 0.7190 - val_loss: 0.6046 - val_acc: 0.7074
Epoch 4/70
160/160 [==============================] - 10s 66ms/step - loss: 0.5952 - acc: 0.7187 - val_loss: 0.5970 - val_acc: 0.7157
Epoch 5/70
160/160 [==============================] - 10s 66ms/step - loss: 0.5967 - acc: 0.7168 - val_loss: 0.5983 - val_acc: 0.7153
Epoch 6/70
160/160 [==============================] - 11s 66ms/step - loss: 0.6000 - acc: 0.7134 - val_loss: 0.5947 - val_acc: 0.720

Epoch 32/70
150/150 [==============================] - 4s 29ms/step - loss: 0.4131 - acc: 0.8271 - val_loss: 0.4215 - val_acc: 0.8178
Epoch 33/70
150/150 [==============================] - 4s 29ms/step - loss: 0.4181 - acc: 0.8214 - val_loss: 0.4019 - val_acc: 0.8300
Epoch 34/70
150/150 [==============================] - 4s 29ms/step - loss: 0.4185 - acc: 0.8197 - val_loss: 0.4249 - val_acc: 0.8172
Epoch 35/70
150/150 [==============================] - 4s 29ms/step - loss: 0.4222 - acc: 0.8182 - val_loss: 0.4258 - val_acc: 0.8078
Epoch 36/70
150/150 [==============================] - 4s 29ms/step - loss: 0.4155 - acc: 0.8206 - val_loss: 0.4088 - val_acc: 0.8244
Epoch 37/70
150/150 [==============================] - 4s 29ms/step - loss: 0.4182 - acc: 0.8180 - val_loss: 0.4117 - val_acc: 0.8178
Epoch 38/70
150/150 [==============================] - 4s 29ms/step - loss: 0.4042 - acc: 0.8277 - val_loss: 0.3979 - val_acc: 0.8275
Epoch 39/70
150/150 [==============================] - 4s 29ms

Epoch 15/90
40/40 [==============================] - 6s 143ms/step - loss: 0.5816 - acc: 0.7069 - val_loss: 0.5678 - val_acc: 0.7183
Epoch 16/90
40/40 [==============================] - 6s 144ms/step - loss: 0.5696 - acc: 0.7190 - val_loss: 0.5728 - val_acc: 0.7105
Epoch 17/90
40/40 [==============================] - 6s 148ms/step - loss: 0.5698 - acc: 0.7141 - val_loss: 0.5565 - val_acc: 0.7221
Epoch 18/90
40/40 [==============================] - 6s 153ms/step - loss: 0.5639 - acc: 0.7189 - val_loss: 0.5662 - val_acc: 0.7115
Epoch 19/90
40/40 [==============================] - 6s 152ms/step - loss: 0.5591 - acc: 0.7221 - val_loss: 0.5551 - val_acc: 0.7175
Epoch 20/90
40/40 [==============================] - 6s 144ms/step - loss: 0.5573 - acc: 0.7170 - val_loss: 0.5502 - val_acc: 0.7185
Epoch 21/90
40/40 [==============================] - 6s 144ms/step - loss: 0.5552 - acc: 0.7151 - val_loss: 0.5462 - val_acc: 0.7147
Epoch 22/90
40/40 [==============================] - 6s 152ms/step - 

...

140/140 [==============================] - 9s 67ms/step - loss: 0.3112 - acc: 0.8687 - val_loss: 0.3272 - val_acc: 0.8737
Epoch 74/80
140/140 [==============================] - 9s 67ms/step - loss: 0.3081 - acc: 0.8706 - val_loss: 0.3332 - val_acc: 0.8594
Epoch 75/80
140/140 [==============================] - 9s 67ms/step - loss: 0.3260 - acc: 0.8644 - val_loss: 0.3397 - val_acc: 0.8606
Epoch 76/80
140/140 [==============================] - 9s 67ms/step - loss: 0.3130 - acc: 0.8691 - val_loss: 0.3197 - val_acc: 0.8694
Epoch 77/80
140/140 [==============================] - 9s 67ms/step - loss: 0.3157 - acc: 0.8669 - val_loss: 0.3554 - val_acc: 0.8438
Epoch 78/80
140/140 [==============================] - 9s 67ms/step - loss: 0.3117 - acc: 0.8702 - val_loss: 0.3213 - val_acc: 0.8575
Epoch 79/80
140/140 [==============================] - 9s 67ms/step - loss: 0.3156 - acc: 0.8655 - val_loss: 0.3333 - val_acc: 0.8663
Epoch 80/80
140/140 [==============================] - 9s 67ms/step - loss

Found 222020 images belonging to 2 classes.
Validation Set:
Found 27752 images belonging to 2 classes.
_________________________________________________________________ 

Model training started...

Epoch 1/70
190/190 [==============================] - 22s 116ms/step - loss: 0.4819 - acc: 0.7795 - val_loss: 0.4178 - val_acc: 0.8229
Epoch 2/70
190/190 [==============================] - 20s 106ms/step - loss: 0.4235 - acc: 0.8188 - val_loss: 0.4183 - val_acc: 0.8135
Epoch 3/70
190/190 [==============================] - 20s 104ms/step - loss: 0.4066 - acc: 0.8259 - val_loss: 0.3973 - val_acc: 0.8241
Epoch 4/70
190/190 [==============================] - 20s 106ms/step - loss: 0.4003 - acc: 0.8289 - val_loss: 0.4113 - val_acc: 0.8244
Epoch 5/70
190/190 [==============================] - 20s 106ms/step - loss: 0.3928 - acc: 0.8322 - val_loss: 0.3973 - val_acc: 0.8306
Epoch 6/70
190/190 [==============================] - 20s 104ms/step - loss: 0.3804 - acc: 0.8358 - val_loss: 0.3803 - val_acc:

Epoch 60/70
190/190 [==============================] - 20s 107ms/step - loss: 0.2993 - acc: 0.8749 - val_loss: 0.3327 - val_acc: 0.8687
Epoch 61/70
190/190 [==============================] - 20s 107ms/step - loss: 0.2983 - acc: 0.8743 - val_loss: 0.3403 - val_acc: 0.8676
Epoch 62/70
190/190 [==============================] - 20s 105ms/step - loss: 0.3037 - acc: 0.8730 - val_loss: 0.3154 - val_acc: 0.8727
Epoch 63/70
190/190 [==============================] - 20s 107ms/step - loss: 0.2960 - acc: 0.8756 - val_loss: 0.3043 - val_acc: 0.8739
Epoch 64/70
190/190 [==============================] - 20s 108ms/step - loss: 0.3013 - acc: 0.8732 - val_loss: 0.3243 - val_acc: 0.8723
Epoch 65/70
190/190 [==============================] - 20s 106ms/step - loss: 0.2987 - acc: 0.8746 - val_loss: 0.3117 - val_acc: 0.8747
Epoch 66/70
190/190 [==============================] - 20s 105ms/step - loss: 0.2953 - acc: 0.8780 - val_loss: 0.3099 - val_acc: 0.8713
Epoch 67/70
190/190 [===========================

Epoch 32/50
40/40 [==============================] - 3s 63ms/step - loss: 0.3712 - acc: 0.8409 - val_loss: 0.3829 - val_acc: 0.8286
Epoch 33/50
40/40 [==============================] - 3s 63ms/step - loss: 0.3771 - acc: 0.8343 - val_loss: 0.3773 - val_acc: 0.8364
Epoch 34/50
40/40 [==============================] - 2s 62ms/step - loss: 0.3826 - acc: 0.8343 - val_loss: 0.4055 - val_acc: 0.8393
Epoch 35/50
40/40 [==============================] - 2s 62ms/step - loss: 0.3914 - acc: 0.8314 - val_loss: 0.4042 - val_acc: 0.8414
Epoch 36/50
40/40 [==============================] - 3s 63ms/step - loss: 0.3859 - acc: 0.8327 - val_loss: 0.3695 - val_acc: 0.8450
Epoch 37/50
40/40 [==============================] - 3s 63ms/step - loss: 0.3740 - acc: 0.8404 - val_loss: 0.4014 - val_acc: 0.8464
Epoch 38/50
40/40 [==============================] - 3s 63ms/step - loss: 0.3642 - acc: 0.8470 - val_loss: 0.4025 - val_acc: 0.8179
Epoch 39/50
40/40 [==============================] - 3s 63ms/step - loss: 0.

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
0,160,60,50,30,0.01000,0.596284,0.717086,0.602438,0.710375,0:02:47,1,True
1,140,200,20,80,0.01000,0.419026,0.819964,0.424821,0.824666,0:14:00,1,True
2,200,30,70,30,0.00010,0.400397,0.828667,0.407492,0.821459,0:02:56,1,True
3,80,190,30,40,0.00100,0.342393,0.854934,0.3493,0.855,0:03:59,1,True
4,100,150,80,90,0.10000,4.61192,0.713867,4.51508,0.719875,0:11:35,1,True
5,200,70,40,40,0.10000,4.57984,0.715857,4.66014,0.710875,0:04:34,1,True
6,60,30,80,40,0.00010,0.414818,0.818889,0.516359,0.784375,0:01:24,1,True
7,200,110,70,60,0.00010,0.351962,0.852273,0.355077,0.848624,0:11:37,1,True
8,160,160,20,70,0.01000,0.598234,0.714492,0.587283,0.726542,0:11:21,1,True
9,120,110,20,40,0.00001,0.415619,0.823788,0.415587,0.819167,0:03:26,1,True


In [9]:
# Keep top performing algorithms
df_fittest = select_fittest(top_percent=10)
df_fittest.nlargest(10, 'val_acc')

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
653,100,190,10,90,0.0001,0.317655,0.865789,0.292571,0.894000,0:11:41,1,True
994,140,180,50,50,0.0010,0.320372,0.864048,0.315853,0.874857,0:10:14,1,True
693,140,160,10,80,0.0010,0.315224,0.867411,0.319296,0.874613,0:13:23,1,True
605,60,200,30,80,0.0010,0.326009,0.861417,0.316879,0.873889,0:19:31,1,True
760,200,190,40,100,0.0001,0.298031,0.874026,0.304660,0.873125,0:39:18,1,True
963,140,200,90,90,0.0001,0.312412,0.867250,0.306135,0.872778,0:34:51,1,True
548,80,200,30,100,0.0010,0.313608,0.866063,0.324402,0.872083,0:21:46,1,True
996,160,140,10,80,0.0010,0.309834,0.872634,0.339324,0.871875,0:12:31,1,True
276,100,170,50,80,0.0001,0.335177,0.857294,0.311656,0.871600,0:24:30,1,True
215,180,130,90,100,0.0010,0.304454,0.868462,0.312375,0.871169,0:37:29,1,True


## 2nd GENERATION ------------------------------------------------------------------

In [10]:
parents = create_parents()
parents

[(276, 936),
 (258, 46),
 (170, 839),
 (968, 246),
 (242, 367),
 (521, 713),
 (853, 302),
 (630, 557),
 (114, 714),
 (605, 980),
 (653, 996),
 (626, 363),
 (256, 591),
 (961, 15),
 (652, 480),
 (215, 760),
 (574, 795),
 (90, 454),
 (93, 88),
 (10, 326),
 (553, 142),
 (418, 266),
 (430, 924),
 (444, 693),
 (781, 496),
 (688, 813),
 (994, 888),
 (982, 892),
 (451, 132),
 (638, 167),
 (95, 269),
 (904, 442),
 (216, 685),
 (355, 158),
 (76, 706),
 (358, 229),
 (801, 548),
 (96, 768),
 (998, 960),
 (716, 632),
 (372, 977),
 (137, 724),
 (832, 176),
 (887, 775),
 (787, 174),
 (275, 938),
 (995, 814),
 (202, 953),
 (437, 163),
 (963, 923)]

In [13]:
# read generations file
df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')
    
for pair in parents:
    df_parents = df_generations.iloc[[pair[0], pair[1]]]
    
    # make children
    df_child_1 = create_child(df_parents, generation=2)
    df_child_2 = create_child(df_parents, generation=2)
    df_child_3 = create_child(df_parents, generation=2)
    df_child_4 = create_child(df_parents, generation=2)
    df_child_5 = create_child(df_parents, generation=2, mutate=True)

    df_children = df_child_1.append([df_child_2, df_child_3, df_child_4, df_child_5], ignore_index=True)

In [14]:
# example of parents
df_parents

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
963,140,200,90,90,0.0001,0.312412,0.867250,0.306135,0.872778,0:34:51,1,True
923,80,110,50,90,0.0001,0.334619,0.858523,0.333084,0.854750,0:07:29,1,True


In [15]:
# example of children
df_children

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
0,80,200,90,90,0.0001,,,,,,2,True
1,140,200,50,90,0.0001,,,,,,2,True
2,80,110,50,90,0.0001,,,,,,2,True
3,80,110,50,90,0.0001,,,,,,2,True
4,140,200,90,90,0.1000,,,,,,2,True


In [7]:
# Train and validate
train_generation(model=model_7, image_size=(50,50), gen=2)

Processing Model 1161 of 1225
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 128)

Epoch 44/100
180/180 [==============================] - 12s 67ms/step - loss: 0.3481 - acc: 0.8529 - val_loss: 0.3345 - val_acc: 0.8580
Epoch 45/100
180/180 [==============================] - 13s 71ms/step - loss: 0.3447 - acc: 0.8512 - val_loss: 0.3444 - val_acc: 0.8583
Epoch 46/100
180/180 [==============================] - 12s 67ms/step - loss: 0.3367 - acc: 0.8560 - val_loss: 0.3822 - val_acc: 0.8411
Epoch 47/100
180/180 [==============================] - 12s 66ms/step - loss: 0.3408 - acc: 0.8559 - val_loss: 0.3451 - val_acc: 0.8552
Epoch 48/100
180/180 [==============================] - 12s 66ms/step - loss: 0.3420 - acc: 0.8549 - val_loss: 0.3609 - val_acc: 0.8470
Epoch 49/100
180/180 [==============================] - 12s 66ms/step - loss: 0.3421 - acc: 0.8543 - val_loss: 0.3676 - val_acc: 0.8389
Epoch 50/100
180/180 [==============================] - 12s 66ms/step - loss: 0.3415 - acc: 0.8553 - val_loss: 0.3462 - val_acc: 0.8485
Epoch 51/100
180/180 [==========================

Found 222020 images belonging to 2 classes.
Validation Set:
Found 27752 images belonging to 2 classes.
_________________________________________________________________ 

Model training started...

Epoch 1/100
180/180 [==============================] - 15s 81ms/step - loss: 0.5761 - acc: 0.7180 - val_loss: 0.5332 - val_acc: 0.7213
Epoch 2/100
180/180 [==============================] - 14s 77ms/step - loss: 0.4450 - acc: 0.7981 - val_loss: 0.4178 - val_acc: 0.8153
Epoch 3/100
180/180 [==============================] - 14s 77ms/step - loss: 0.4120 - acc: 0.8236 - val_loss: 0.4205 - val_acc: 0.8191
Epoch 4/100
180/180 [==============================] - 14s 78ms/step - loss: 0.4120 - acc: 0.8233 - val_loss: 0.4090 - val_acc: 0.8293
Epoch 5/100
180/180 [==============================] - 15s 81ms/step - loss: 0.4047 - acc: 0.8257 - val_loss: 0.3949 - val_acc: 0.8290
Epoch 6/100
180/180 [==============================] - 9s 48ms/step - loss: 0.4038 - acc: 0.8290 - val_loss: 0.4056 - val_acc: 

Epoch 60/100
180/180 [==============================] - 14s 76ms/step - loss: 0.3271 - acc: 0.8633 - val_loss: 0.3419 - val_acc: 0.8581
Epoch 61/100
180/180 [==============================] - 14s 76ms/step - loss: 0.3363 - acc: 0.8584 - val_loss: 0.3213 - val_acc: 0.8652
Epoch 62/100
180/180 [==============================] - 14s 77ms/step - loss: 0.3322 - acc: 0.8589 - val_loss: 0.3362 - val_acc: 0.8554
Epoch 63/100
180/180 [==============================] - 14s 78ms/step - loss: 0.3293 - acc: 0.8588 - val_loss: 0.3515 - val_acc: 0.8468
Epoch 64/100
180/180 [==============================] - 14s 78ms/step - loss: 0.3334 - acc: 0.8571 - val_loss: 0.3227 - val_acc: 0.8664
Epoch 65/100
180/180 [==============================] - 14s 78ms/step - loss: 0.3245 - acc: 0.8633 - val_loss: 0.3187 - val_acc: 0.8659
Epoch 66/100
180/180 [==============================] - 14s 78ms/step - loss: 0.3273 - acc: 0.8612 - val_loss: 0.3322 - val_acc: 0.8667
Epoch 67/100
180/180 [==========================

150/150 [==============================] - 11s 72ms/step - loss: 4.6226 - acc: 0.7105 - val_loss: 4.6858 - val_acc: 0.7093
Epoch 2/50
150/150 [==============================] - 10s 68ms/step - loss: 4.5653 - acc: 0.7168 - val_loss: 4.5887 - val_acc: 0.7153
Epoch 3/50
150/150 [==============================] - 10s 68ms/step - loss: 4.5422 - acc: 0.7182 - val_loss: 4.4403 - val_acc: 0.7245
Epoch 4/50
150/150 [==============================] - 10s 68ms/step - loss: 4.5215 - acc: 0.7195 - val_loss: 4.6117 - val_acc: 0.7139
Epoch 5/50
150/150 [==============================] - 10s 68ms/step - loss: 4.5484 - acc: 0.7178 - val_loss: 4.6150 - val_acc: 0.7137
Epoch 6/50
150/150 [==============================] - 10s 68ms/step - loss: 4.5714 - acc: 0.7164 - val_loss: 4.4586 - val_acc: 0.7234
Epoch 7/50
150/150 [==============================] - 10s 69ms/step - loss: 4.6059 - acc: 0.7142 - val_loss: 4.5229 - val_acc: 0.7194
Epoch 8/50
150/150 [==============================] - 10s 69ms/step - los

...

200/200 [==============================] - 14s 69ms/step - loss: 0.3158 - acc: 0.8680 - val_loss: 0.3196 - val_acc: 0.8664
Epoch 79/90
200/200 [==============================] - 14s 69ms/step - loss: 0.3118 - acc: 0.8665 - val_loss: 0.3089 - val_acc: 0.8681
Epoch 80/90
200/200 [==============================] - 14s 69ms/step - loss: 0.3110 - acc: 0.8703 - val_loss: 0.3104 - val_acc: 0.8684
Epoch 81/90
200/200 [==============================] - 14s 68ms/step - loss: 0.3131 - acc: 0.8685 - val_loss: 0.3122 - val_acc: 0.8664
Epoch 82/90
200/200 [==============================] - 14s 68ms/step - loss: 0.3051 - acc: 0.8738 - val_loss: 0.3201 - val_acc: 0.8669
Epoch 83/90
200/200 [==============================] - 14s 68ms/step - loss: 0.3035 - acc: 0.8716 - val_loss: 0.3320 - val_acc: 0.8559
Epoch 84/90
200/200 [==============================] - 14s 68ms/step - loss: 0.3123 - acc: 0.8688 - val_loss: 0.3149 - val_acc: 0.8704
Epoch 85/90
200/200 [==============================] - 13s 67ms/ste

Epoch 30/90
110/110 [==============================] - 5s 45ms/step - loss: 0.3837 - acc: 0.8360 - val_loss: 0.3883 - val_acc: 0.8305
Epoch 31/90
110/110 [==============================] - 5s 45ms/step - loss: 0.3902 - acc: 0.8335 - val_loss: 0.3964 - val_acc: 0.8290
Epoch 32/90
110/110 [==============================] - 5s 45ms/step - loss: 0.3843 - acc: 0.8331 - val_loss: 0.3822 - val_acc: 0.8352
Epoch 33/90
110/110 [==============================] - 5s 46ms/step - loss: 0.3879 - acc: 0.8349 - val_loss: 0.3957 - val_acc: 0.8318
Epoch 34/90
110/110 [==============================] - 5s 46ms/step - loss: 0.3839 - acc: 0.8303 - val_loss: 0.3768 - val_acc: 0.8347
Epoch 35/90
110/110 [==============================] - 5s 46ms/step - loss: 0.3836 - acc: 0.8349 - val_loss: 0.3821 - val_acc: 0.8359
Epoch 36/90
110/110 [==============================] - 5s 45ms/step - loss: 0.3884 - acc: 0.8325 - val_loss: 0.4097 - val_acc: 0.8267
Epoch 37/90
110/110 [==============================] - 5s 46ms

Model saved as: " MODELS / model_breast_cancer_1223.h5 "
######################################################################################################################## 

Processing Model 1225 of 1225
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_257 (Conv2D)          (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d_257 (MaxPoolin (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_258 (Conv2D)          (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_258 (MaxPoolin (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_259 (Conv2D)          (None, 9, 9, 128)         73856     
________________________________________________________________

Epoch 43/90
200/200 [==============================] - 16s 80ms/step - loss: 4.4969 - acc: 0.7210 - val_loss: 4.6811 - val_acc: 0.7096
Epoch 44/90
200/200 [==============================] - 16s 78ms/step - loss: 4.5770 - acc: 0.7160 - val_loss: 4.4606 - val_acc: 0.7233
Epoch 45/90
200/200 [==============================] - 16s 78ms/step - loss: 4.5873 - acc: 0.7154 - val_loss: 4.5753 - val_acc: 0.7161
Epoch 46/90
200/200 [==============================] - 16s 79ms/step - loss: 4.5960 - acc: 0.7149 - val_loss: 4.6538 - val_acc: 0.7113
Epoch 47/90
200/200 [==============================] - 16s 79ms/step - loss: 4.5706 - acc: 0.7164 - val_loss: 4.4592 - val_acc: 0.7233
Epoch 48/90
200/200 [==============================] - 16s 79ms/step - loss: 4.5476 - acc: 0.7179 - val_loss: 4.6167 - val_acc: 0.7136
Epoch 49/90
200/200 [==============================] - 16s 78ms/step - loss: 4.5956 - acc: 0.7149 - val_loss: 4.6308 - val_acc: 0.7127
Epoch 50/90
200/200 [==============================] - 

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
0,160,60,50,30,0.01000,0.596284,0.717086,0.602438,0.710375,0:02:47,1,False
1,140,200,20,80,0.01000,0.419026,0.819964,0.424821,0.824666,0:14:00,1,False
2,200,30,70,30,0.00010,0.400397,0.828667,0.407492,0.821459,0:02:56,1,False
3,80,190,30,40,0.00100,0.342393,0.854934,0.3493,0.855,0:03:59,1,False
4,100,150,80,90,0.10000,4.61192,0.713867,4.51508,0.719875,0:11:35,1,False
5,200,70,40,40,0.10000,4.57984,0.715857,4.66014,0.710875,0:04:34,1,False
6,60,30,80,40,0.00010,0.414818,0.818889,0.516359,0.784375,0:01:24,1,False
7,200,110,70,60,0.00010,0.351962,0.852273,0.355077,0.848624,0:11:37,1,False
8,160,160,20,70,0.01000,0.598234,0.714492,0.587283,0.726542,0:11:21,1,False
9,120,110,20,40,0.00001,0.415619,0.823788,0.415587,0.819167,0:03:26,1,False


In [10]:
# Keep top performing algorithms
df_fittest = select_fittest(top_percent=10)
df_fittest.nlargest(10, 'val_acc')

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
653,100,190,10,90,0.0001,0.317655,0.865789,0.292571,0.894000,0:11:41,1,True
1178,80,190,30,50,0.0010,0.333464,0.858750,0.292957,0.884583,0:05:13,2,True
1129,200,200,20,70,0.0001,0.311461,0.867375,0.295427,0.881832,0:29:50,2,True
1107,80,160,10,80,0.0010,0.321133,0.866250,0.336637,0.881250,0:10:42,2,True
1038,100,200,10,90,0.0010,0.306632,0.870800,0.317246,0.881000,0:14:31,2,True
1039,100,190,10,50,0.0001,0.344512,0.852526,0.317177,0.878000,0:07:40,2,True
1029,200,150,20,90,0.0010,0.294208,0.877200,0.308905,0.877500,0:21:22,2,True
1106,140,200,10,80,0.0001,0.316372,0.865000,0.288847,0.876935,0:21:26,2,True
1047,160,140,10,80,0.0010,0.301332,0.874420,0.303735,0.876875,0:14:22,2,True
1125,200,120,90,100,0.0010,0.299698,0.874125,0.298709,0.876500,0:40:10,2,True


## 3rd GENERATION ------------------------------------------------------------------

In [11]:
parents = create_parents()
parents

[(1150, 1074),
 (994, 693),
 (1083, 653),
 (1178, 1047),
 (996, 1067),
 (1068, 605),
 (1106, 1016),
 (1039, 1119),
 (1142, 963),
 (760, 548),
 (1125, 1107),
 (1032, 1134),
 (1029, 1103),
 (1176, 1171),
 (1038, 1129),
 (276, 1112)]

In [12]:
# read generations file
df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')
    
for pair in parents:
    df_parents = df_generations.iloc[[pair[0], pair[1]]]
    
    # make children
    df_child_1 = create_child(df_parents, generation=3)
    df_child_2 = create_child(df_parents, generation=3)
    df_child_3 = create_child(df_parents, generation=3)
    df_child_4 = create_child(df_parents, generation=3)
    df_child_5 = create_child(df_parents, generation=3, mutate=True)

    df_children = df_child_1.append([df_child_2, df_child_3, df_child_4, df_child_5], ignore_index=True)

In [13]:
df_parents

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
276,100,170,50,80,0.0001,0.335177,0.857294,0.311656,0.871600,0:24:30,1,True
1112,180,160,60,100,0.0010,0.293139,0.878368,0.306576,0.873426,0:34:54,2,True


In [14]:
df_children

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
0,180,170,50,80,0.0001,,,,,,3,True
1,180,170,60,80,0.0010,,,,,,3,True
2,180,160,50,100,0.0010,,,,,,3,True
3,100,160,60,80,0.0010,,,,,,3,True
4,180,10,50,100,0.0010,,,,,,3,True


In [15]:
# Train and validate
train_generation(model=model_7, image_size=(50,50), gen=3)

Processing Model 1226 of 1299
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_261 (Conv2D)          (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d_261 (MaxPoolin (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_262 (Conv2D)          (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_262 (MaxPoolin (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_263 (Conv2D)          (None, 9, 9, 128)         73856     
_________________________________________________________________
max_pooling2d_263 (MaxPoolin (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_264 (Conv2D)          (None, 2, 2, 128)

190/190 [==============================] - 17s 89ms/step - loss: 0.3141 - acc: 0.8681 - val_loss: 0.3371 - val_acc: 0.8625
Epoch 45/90
190/190 [==============================] - 17s 90ms/step - loss: 0.3108 - acc: 0.8706 - val_loss: 0.3181 - val_acc: 0.8665
Epoch 46/90
190/190 [==============================] - 17s 90ms/step - loss: 0.3064 - acc: 0.8703 - val_loss: 0.3213 - val_acc: 0.8651
Epoch 47/90
190/190 [==============================] - 17s 89ms/step - loss: 0.3043 - acc: 0.8726 - val_loss: 0.3296 - val_acc: 0.8630
Epoch 48/90
190/190 [==============================] - 17s 89ms/step - loss: 0.3056 - acc: 0.8727 - val_loss: 0.3265 - val_acc: 0.8656
Epoch 49/90
190/190 [==============================] - 18s 95ms/step - loss: 0.3065 - acc: 0.8703 - val_loss: 0.3251 - val_acc: 0.8612
Epoch 50/90
190/190 [==============================] - 18s 97ms/step - loss: 0.2986 - acc: 0.8754 - val_loss: 0.3170 - val_acc: 0.8625
Epoch 51/90
190/190 [==============================] - 18s 93ms/ste

Found 222020 images belonging to 2 classes.
Validation Set:
Found 27752 images belonging to 2 classes.
_________________________________________________________________ 

Model training started...

Epoch 1/100
90/90 [==============================] - 12s 134ms/step - loss: 0.5131 - acc: 0.7492 - val_loss: 0.4500 - val_acc: 0.8005
Epoch 2/100
90/90 [==============================] - 8s 84ms/step - loss: 0.4366 - acc: 0.8146 - val_loss: 0.4211 - val_acc: 0.8303
Epoch 3/100
90/90 [==============================] - 8s 87ms/step - loss: 0.4174 - acc: 0.8231 - val_loss: 0.4109 - val_acc: 0.8143
Epoch 4/100
90/90 [==============================] - 8s 86ms/step - loss: 0.4167 - acc: 0.8192 - val_loss: 0.4000 - val_acc: 0.8297
Epoch 5/100
90/90 [==============================] - 8s 86ms/step - loss: 0.4050 - acc: 0.8279 - val_loss: 0.3993 - val_acc: 0.8285
Epoch 6/100
90/90 [==============================] - 8s 85ms/step - loss: 0.4055 - acc: 0.8273 - val_loss: 0.4193 - val_acc: 0.8105
Epoch 7/

90/90 [==============================] - 8s 87ms/step - loss: 0.3200 - acc: 0.8666 - val_loss: 0.3538 - val_acc: 0.8590
Epoch 62/100
90/90 [==============================] - 8s 88ms/step - loss: 0.3246 - acc: 0.8624 - val_loss: 0.3321 - val_acc: 0.8770
Epoch 63/100
90/90 [==============================] - 8s 88ms/step - loss: 0.3235 - acc: 0.8658 - val_loss: 0.3140 - val_acc: 0.8682
Epoch 64/100
90/90 [==============================] - 8s 87ms/step - loss: 0.3227 - acc: 0.8655 - val_loss: 0.3513 - val_acc: 0.8633
Epoch 65/100
90/90 [==============================] - 8s 88ms/step - loss: 0.3225 - acc: 0.8656 - val_loss: 0.3328 - val_acc: 0.8682
Epoch 66/100
90/90 [==============================] - 8s 88ms/step - loss: 0.3140 - acc: 0.8647 - val_loss: 0.3222 - val_acc: 0.8688
Epoch 67/100
90/90 [==============================] - 8s 90ms/step - loss: 0.3315 - acc: 0.8594 - val_loss: 0.3437 - val_acc: 0.8510
Epoch 68/100
90/90 [==============================] - 8s 89ms/step - loss: 0.3225 

190/190 [==============================] - 16s 82ms/step - loss: 0.3933 - acc: 0.8314 - val_loss: 0.3777 - val_acc: 0.8347
Epoch 4/90
190/190 [==============================] - 16s 82ms/step - loss: 0.3887 - acc: 0.8331 - val_loss: 0.3777 - val_acc: 0.8315
Epoch 5/90
190/190 [==============================] - 16s 82ms/step - loss: 0.3783 - acc: 0.8374 - val_loss: 0.3875 - val_acc: 0.8285
Epoch 6/90
190/190 [==============================] - 16s 82ms/step - loss: 0.3800 - acc: 0.8376 - val_loss: 0.4155 - val_acc: 0.8310
Epoch 7/90
190/190 [==============================] - 16s 83ms/step - loss: 0.3727 - acc: 0.8387 - val_loss: 0.3747 - val_acc: 0.8456
Epoch 8/90
190/190 [==============================] - 16s 84ms/step - loss: 0.3619 - acc: 0.8459 - val_loss: 0.3888 - val_acc: 0.8403
Epoch 9/90
190/190 [==============================] - 16s 84ms/step - loss: 0.3595 - acc: 0.8465 - val_loss: 0.3653 - val_acc: 0.8493
Epoch 10/90
190/190 [==============================] - 16s 83ms/step - lo

...

Found 27752 images belonging to 2 classes.
_________________________________________________________________ 

Model training started...

Epoch 1/80
160/160 [==============================] - 20s 123ms/step - loss: 0.4935 - acc: 0.7744 - val_loss: 0.4456 - val_acc: 0.8197
Epoch 2/80
160/160 [==============================] - 10s 65ms/step - loss: 0.4277 - acc: 0.8184 - val_loss: 0.4266 - val_acc: 0.8097
Epoch 3/80
160/160 [==============================] - 10s 64ms/step - loss: 0.4105 - acc: 0.8276 - val_loss: 0.4142 - val_acc: 0.8235
Epoch 4/80
160/160 [==============================] - 10s 65ms/step - loss: 0.4124 - acc: 0.8234 - val_loss: 0.4222 - val_acc: 0.8232
Epoch 5/80
160/160 [==============================] - 10s 65ms/step - loss: 0.3975 - acc: 0.8331 - val_loss: 0.3935 - val_acc: 0.8303
Epoch 6/80
160/160 [==============================] - 10s 65ms/step - loss: 0.4005 - acc: 0.8259 - val_loss: 0.3835 - val_acc: 0.8295
Epoch 7/80
160/160 [==============================] - 10s

Epoch 61/80
160/160 [==============================] - 10s 63ms/step - loss: 0.3203 - acc: 0.8629 - val_loss: 0.3780 - val_acc: 0.8345
Epoch 62/80
160/160 [==============================] - 10s 63ms/step - loss: 0.3361 - acc: 0.8579 - val_loss: 0.3483 - val_acc: 0.8603
Epoch 63/80
160/160 [==============================] - 10s 63ms/step - loss: 0.3303 - acc: 0.8602 - val_loss: 0.3319 - val_acc: 0.8632
Epoch 64/80
160/160 [==============================] - 10s 63ms/step - loss: 0.3215 - acc: 0.8642 - val_loss: 0.3282 - val_acc: 0.8633
Epoch 65/80
160/160 [==============================] - 10s 63ms/step - loss: 0.3197 - acc: 0.8633 - val_loss: 0.3224 - val_acc: 0.8683
Epoch 66/80
160/160 [==============================] - 10s 63ms/step - loss: 0.3289 - acc: 0.8619 - val_loss: 0.3368 - val_acc: 0.8618
Epoch 67/80
160/160 [==============================] - 10s 63ms/step - loss: 0.3178 - acc: 0.8634 - val_loss: 0.3119 - val_acc: 0.8750
Epoch 68/80
160/160 [==============================] - 

10/10 [==============================] - 4s 432ms/step - loss: 0.4316 - acc: 0.8128 - val_loss: 0.4168 - val_acc: 0.8209
Epoch 23/100
10/10 [==============================] - 4s 444ms/step - loss: 0.4014 - acc: 0.8300 - val_loss: 0.4036 - val_acc: 0.8277
Epoch 24/100
10/10 [==============================] - 4s 444ms/step - loss: 0.3798 - acc: 0.8339 - val_loss: 0.4128 - val_acc: 0.8130
Epoch 25/100
10/10 [==============================] - 4s 437ms/step - loss: 0.4178 - acc: 0.8183 - val_loss: 0.4118 - val_acc: 0.8232
Epoch 26/100
10/10 [==============================] - 4s 427ms/step - loss: 0.3935 - acc: 0.8339 - val_loss: 0.4426 - val_acc: 0.8017
Epoch 27/100
10/10 [==============================] - 4s 427ms/step - loss: 0.4095 - acc: 0.8372 - val_loss: 0.4708 - val_acc: 0.8074
Epoch 28/100
10/10 [==============================] - 4s 431ms/step - loss: 0.4278 - acc: 0.8200 - val_loss: 0.4403 - val_acc: 0.8251
Epoch 29/100
10/10 [==============================] - 4s 443ms/step - loss:

10/10 [==============================] - 4s 429ms/step - loss: 0.3987 - acc: 0.8367 - val_loss: 0.3889 - val_acc: 0.8243
Epoch 84/100
10/10 [==============================] - 4s 434ms/step - loss: 0.3874 - acc: 0.8367 - val_loss: 0.3909 - val_acc: 0.8251
Epoch 85/100
10/10 [==============================] - 4s 446ms/step - loss: 0.3880 - acc: 0.8222 - val_loss: 0.3993 - val_acc: 0.8193
Epoch 86/100
10/10 [==============================] - 4s 443ms/step - loss: 0.3884 - acc: 0.8272 - val_loss: 0.3868 - val_acc: 0.8356
Epoch 87/100
10/10 [==============================] - 4s 442ms/step - loss: 0.3525 - acc: 0.8550 - val_loss: 0.3791 - val_acc: 0.8369
Epoch 88/100
10/10 [==============================] - 4s 436ms/step - loss: 0.3648 - acc: 0.8406 - val_loss: 0.3884 - val_acc: 0.8316
Epoch 89/100
10/10 [==============================] - 4s 434ms/step - loss: 0.3852 - acc: 0.8350 - val_loss: 0.3778 - val_acc: 0.8442
Epoch 90/100
10/10 [==============================] - 4s 436ms/step - loss:

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
0,160,60,50,30,0.01000,0.596284,0.717086,0.602438,0.710375,0:02:47,1,False
1,140,200,20,80,0.01000,0.419026,0.819964,0.424821,0.824666,0:14:00,1,False
2,200,30,70,30,0.00010,0.400397,0.828667,0.407492,0.821459,0:02:56,1,False
3,80,190,30,40,0.00100,0.342393,0.854934,0.3493,0.855,0:03:59,1,False
4,100,150,80,90,0.10000,4.61192,0.713867,4.51508,0.719875,0:11:35,1,False
5,200,70,40,40,0.10000,4.57984,0.715857,4.66014,0.710875,0:04:34,1,False
6,60,30,80,40,0.00010,0.414818,0.818889,0.516359,0.784375,0:01:24,1,False
7,200,110,70,60,0.00010,0.351962,0.852273,0.355077,0.848624,0:11:37,1,False
8,160,160,20,70,0.01000,0.598234,0.714492,0.587283,0.726542,0:11:21,1,False
9,120,110,20,40,0.00001,0.415619,0.823788,0.415587,0.819167,0:03:26,1,False


In [16]:
# Keep top performing algorithms
df_fittest = select_fittest(top_percent=10)
df_fittest.nlargest(10, 'val_acc')

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
653,100,190,10,90,0.0001,0.317655,0.865789,0.292571,0.894000,0:11:41,1,True
1240,160,140,10,80,0.0010,0.309508,0.870714,0.298903,0.886250,0:12:35,3,True
1178,80,190,30,50,0.0010,0.333464,0.858750,0.292957,0.884583,0:05:13,2,True
1278,200,130,10,90,0.0010,0.302942,0.873192,0.296192,0.882000,0:17:44,3,True
1129,200,200,20,70,0.0001,0.311461,0.867375,0.295427,0.881832,0:29:50,2,True
1107,80,160,10,80,0.0010,0.321133,0.866250,0.336637,0.881250,0:10:42,2,True
1038,100,200,10,90,0.0010,0.306632,0.870800,0.317246,0.881000,0:14:31,2,True
1250,200,190,30,100,0.0010,0.284835,0.881026,0.298489,0.878500,0:28:42,3,True
1267,200,150,20,60,0.0010,0.310335,0.870633,0.298475,0.878250,0:13:40,3,True
1039,100,190,10,50,0.0001,0.344512,0.852526,0.317177,0.878000,0:07:40,2,True


## 4th GENERATION ------------------------------------------------------------------

In [17]:
parents = create_parents()
parents

[(1039, 1278), (1250, 1129), (1107, 1267), (653, 1029), (1178, 1240)]

In [18]:
# read generations file
df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')
    
for pair in parents:
    df_parents = df_generations.iloc[[pair[0], pair[1]]]
    
    # make children
    df_child_1 = create_child(df_parents, generation=4)
    df_child_2 = create_child(df_parents, generation=4)
    df_child_3 = create_child(df_parents, generation=4)
    df_child_4 = create_child(df_parents, generation=4)
    df_child_5 = create_child(df_parents, generation=4, mutate=True)

    df_children = df_child_1.append([df_child_2, df_child_3, df_child_4, df_child_5], ignore_index=True)

In [19]:
df_parents

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
1178,80,190,30,50,0.001,0.333464,0.858750,0.292957,0.884583,0:05:13,2,True
1240,160,140,10,80,0.001,0.309508,0.870714,0.298903,0.886250,0:12:35,3,True


In [20]:
df_children

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
0,160,190,30,80,0.001,,,,,,4,True
1,80,140,10,80,0.001,,,,,,4,True
2,80,190,30,80,0.001,,,,,,4,True
3,160,190,10,50,0.001,,,,,,4,True
4,80,120,30,80,0.001,,,,,,4,True


In [21]:
# Train and validate
train_generation(model=model_7, image_size=(50,50), gen=4)

Processing Model 1300 of 1323
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_557 (Conv2D)          (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d_557 (MaxPoolin (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_558 (Conv2D)          (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_558 (MaxPoolin (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_559 (Conv2D)          (None, 9, 9, 128)         73856     
_________________________________________________________________
max_pooling2d_559 (MaxPoolin (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_560 (Conv2D)          (None, 2, 2, 128)

130/130 [==============================] - 7s 51ms/step - loss: 0.3787 - acc: 0.8363 - val_loss: 0.4000 - val_acc: 0.8100
Epoch 45/90
130/130 [==============================] - 7s 52ms/step - loss: 0.3718 - acc: 0.8392 - val_loss: 0.3517 - val_acc: 0.8470
Epoch 46/90
130/130 [==============================] - 7s 51ms/step - loss: 0.3632 - acc: 0.8442 - val_loss: 0.3774 - val_acc: 0.8370
Epoch 47/90
130/130 [==============================] - 7s 50ms/step - loss: 0.3797 - acc: 0.8361 - val_loss: 0.3450 - val_acc: 0.8490
Epoch 48/90
130/130 [==============================] - 7s 51ms/step - loss: 0.3541 - acc: 0.8520 - val_loss: 0.3592 - val_acc: 0.8420
Epoch 49/90
130/130 [==============================] - 7s 51ms/step - loss: 0.3563 - acc: 0.8492 - val_loss: 0.3704 - val_acc: 0.8470
Epoch 50/90
130/130 [==============================] - 7s 54ms/step - loss: 0.3593 - acc: 0.8460 - val_loss: 0.3676 - val_acc: 0.8490
Epoch 51/90
130/130 [==============================] - 7s 56ms/step - loss

Found 222020 images belonging to 2 classes.
Validation Set:
Found 27752 images belonging to 2 classes.
_________________________________________________________________ 

Model training started...

Epoch 1/90
190/190 [==============================] - 28s 149ms/step - loss: 0.5620 - acc: 0.7190 - val_loss: 0.4758 - val_acc: 0.8030
Epoch 2/90
190/190 [==============================] - 17s 92ms/step - loss: 0.4294 - acc: 0.8143 - val_loss: 0.4157 - val_acc: 0.8195
Epoch 3/90
190/190 [==============================] - 19s 97ms/step - loss: 0.4201 - acc: 0.8199 - val_loss: 0.4268 - val_acc: 0.8140
Epoch 4/90
190/190 [==============================] - 18s 94ms/step - loss: 0.4168 - acc: 0.8223 - val_loss: 0.4045 - val_acc: 0.8215
Epoch 5/90
190/190 [==============================] - 18s 97ms/step - loss: 0.4031 - acc: 0.8282 - val_loss: 0.4146 - val_acc: 0.8175
Epoch 6/90
190/190 [==============================] - 19s 101ms/step - loss: 0.4071 - acc: 0.8266 - val_loss: 0.3997 - val_acc: 0.8

190/190 [==============================] - 18s 94ms/step - loss: 0.3260 - acc: 0.8596 - val_loss: 0.3339 - val_acc: 0.8540
Epoch 61/90
190/190 [==============================] - 18s 93ms/step - loss: 0.3244 - acc: 0.8628 - val_loss: 0.3231 - val_acc: 0.8650
Epoch 62/90
190/190 [==============================] - 18s 93ms/step - loss: 0.3183 - acc: 0.8656 - val_loss: 0.3352 - val_acc: 0.8545
Epoch 63/90
190/190 [==============================] - 18s 94ms/step - loss: 0.3226 - acc: 0.8641 - val_loss: 0.3283 - val_acc: 0.8575
Epoch 64/90
190/190 [==============================] - 18s 93ms/step - loss: 0.3211 - acc: 0.8641 - val_loss: 0.3102 - val_acc: 0.8705
Epoch 65/90
190/190 [==============================] - 18s 93ms/step - loss: 0.3212 - acc: 0.8650 - val_loss: 0.3242 - val_acc: 0.8615
Epoch 66/90
190/190 [==============================] - 17s 92ms/step - loss: 0.3153 - acc: 0.8653 - val_loss: 0.3333 - val_acc: 0.8600
Epoch 67/90
190/190 [==============================] - 17s 92ms/ste

130/130 [==============================] - 12s 94ms/step - loss: 0.3918 - acc: 0.8331 - val_loss: 0.3774 - val_acc: 0.8345
Epoch 12/50
130/130 [==============================] - 12s 94ms/step - loss: 0.3896 - acc: 0.8332 - val_loss: 0.3951 - val_acc: 0.8320
Epoch 13/50
130/130 [==============================] - 12s 94ms/step - loss: 0.3780 - acc: 0.8369 - val_loss: 0.3803 - val_acc: 0.8435
Epoch 14/50
130/130 [==============================] - 12s 95ms/step - loss: 0.3790 - acc: 0.8359 - val_loss: 0.3783 - val_acc: 0.8350
Epoch 15/50
130/130 [==============================] - 12s 95ms/step - loss: 0.3762 - acc: 0.8367 - val_loss: 0.3829 - val_acc: 0.8390
Epoch 16/50
130/130 [==============================] - 12s 95ms/step - loss: 0.3857 - acc: 0.8333 - val_loss: 0.3889 - val_acc: 0.8300
Epoch 17/50
130/130 [==============================] - 12s 95ms/step - loss: 0.3756 - acc: 0.8380 - val_loss: 0.3997 - val_acc: 0.8235
Epoch 18/50
130/130 [==============================] - 13s 97ms/ste

...

190/190 [==============================] - 9s 48ms/step - loss: 0.3252 - acc: 0.8636 - val_loss: 0.3466 - val_acc: 0.8479
Epoch 63/80
190/190 [==============================] - 9s 47ms/step - loss: 0.3203 - acc: 0.8643 - val_loss: 0.3645 - val_acc: 0.8412
Epoch 64/80
190/190 [==============================] - 9s 48ms/step - loss: 0.3199 - acc: 0.8641 - val_loss: 0.3322 - val_acc: 0.8613
Epoch 65/80
190/190 [==============================] - 9s 47ms/step - loss: 0.3264 - acc: 0.8598 - val_loss: 0.3587 - val_acc: 0.8587
Epoch 66/80
190/190 [==============================] - 9s 47ms/step - loss: 0.3233 - acc: 0.8662 - val_loss: 0.3453 - val_acc: 0.8617
Epoch 67/80
190/190 [==============================] - 9s 48ms/step - loss: 0.3294 - acc: 0.8601 - val_loss: 0.3314 - val_acc: 0.8675
Epoch 68/80
190/190 [==============================] - 9s 48ms/step - loss: 0.3298 - acc: 0.8591 - val_loss: 0.3395 - val_acc: 0.8642
Epoch 69/80
190/190 [==============================] - 9s 48ms/step - loss

190/190 [==============================] - 15s 80ms/step - loss: 0.3397 - acc: 0.8551 - val_loss: 0.3599 - val_acc: 0.8569
Epoch 24/50
190/190 [==============================] - 15s 80ms/step - loss: 0.3384 - acc: 0.8563 - val_loss: 0.3644 - val_acc: 0.8544
Epoch 25/50
190/190 [==============================] - 15s 80ms/step - loss: 0.3374 - acc: 0.8559 - val_loss: 0.3403 - val_acc: 0.8637
Epoch 26/50
190/190 [==============================] - 15s 80ms/step - loss: 0.3368 - acc: 0.8568 - val_loss: 0.3261 - val_acc: 0.8669
Epoch 27/50
190/190 [==============================] - 15s 80ms/step - loss: 0.3332 - acc: 0.8567 - val_loss: 0.3484 - val_acc: 0.8375
Epoch 28/50
190/190 [==============================] - 15s 80ms/step - loss: 0.3350 - acc: 0.8571 - val_loss: 0.3494 - val_acc: 0.8587
Epoch 29/50
190/190 [==============================] - 15s 80ms/step - loss: 0.3311 - acc: 0.8587 - val_loss: 0.3447 - val_acc: 0.8531
Epoch 30/50
190/190 [==============================] - 15s 81ms/ste

120/120 [==============================] - 6s 51ms/step - loss: 0.3905 - acc: 0.8335 - val_loss: 0.3999 - val_acc: 0.8233
Epoch 15/80
120/120 [==============================] - 6s 50ms/step - loss: 0.3822 - acc: 0.8366 - val_loss: 0.3704 - val_acc: 0.8396
Epoch 16/80
120/120 [==============================] - 6s 51ms/step - loss: 0.3875 - acc: 0.8330 - val_loss: 0.3799 - val_acc: 0.8392
Epoch 17/80
120/120 [==============================] - 6s 50ms/step - loss: 0.3995 - acc: 0.8310 - val_loss: 0.3908 - val_acc: 0.8371
Epoch 18/80
120/120 [==============================] - 6s 50ms/step - loss: 0.3897 - acc: 0.8343 - val_loss: 0.3682 - val_acc: 0.8483
Epoch 19/80
120/120 [==============================] - 6s 51ms/step - loss: 0.3838 - acc: 0.8390 - val_loss: 0.3683 - val_acc: 0.8442
Epoch 20/80
120/120 [==============================] - 6s 51ms/step - loss: 0.3709 - acc: 0.8445 - val_loss: 0.3999 - val_acc: 0.8367
Epoch 21/80
120/120 [==============================] - 6s 50ms/step - loss

120/120 [==============================] - 6s 51ms/step - loss: 0.3346 - acc: 0.8568 - val_loss: 0.3303 - val_acc: 0.8646
Epoch 76/80
120/120 [==============================] - 6s 51ms/step - loss: 0.3319 - acc: 0.8590 - val_loss: 0.3613 - val_acc: 0.8467
Epoch 77/80
120/120 [==============================] - 6s 51ms/step - loss: 0.3282 - acc: 0.8608 - val_loss: 0.3207 - val_acc: 0.8646
Epoch 78/80
120/120 [==============================] - 6s 51ms/step - loss: 0.3312 - acc: 0.8566 - val_loss: 0.3461 - val_acc: 0.8550
Epoch 79/80
120/120 [==============================] - 6s 51ms/step - loss: 0.3362 - acc: 0.8544 - val_loss: 0.3530 - val_acc: 0.8521
Epoch 80/80
120/120 [==============================] - 6s 51ms/step - loss: 0.3238 - acc: 0.8613 - val_loss: 0.3218 - val_acc: 0.8654

Training process completed in: 0 h 8 m 19 s
Model saved as: " MODELS / model_breast_cancer_1322.h5 "
##########################################################################################################

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
0,160,60,50,30,0.01000,0.596284,0.717086,0.602438,0.710375,0:02:47,1,False
1,140,200,20,80,0.01000,0.419026,0.819964,0.424821,0.824666,0:14:00,1,False
2,200,30,70,30,0.00010,0.400397,0.828667,0.407492,0.821459,0:02:56,1,False
3,80,190,30,40,0.00100,0.342393,0.854934,0.3493,0.855,0:03:59,1,False
4,100,150,80,90,0.10000,4.61192,0.713867,4.51508,0.719875,0:11:35,1,False
5,200,70,40,40,0.10000,4.57984,0.715857,4.66014,0.710875,0:04:34,1,False
6,60,30,80,40,0.00010,0.414818,0.818889,0.516359,0.784375,0:01:24,1,False
7,200,110,70,60,0.00010,0.351962,0.852273,0.355077,0.848624,0:11:37,1,False
8,160,160,20,70,0.01000,0.598234,0.714492,0.587283,0.726542,0:11:21,1,False
9,120,110,20,40,0.00001,0.415619,0.823788,0.415587,0.819167,0:03:26,1,False


In [22]:
# Get number of current live algorithms
df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')
len(df_generations[df_generations.alive])

35

In [23]:
# Keep top performing algorithms
df_fittest = select_fittest(top_percent=50)
df_fittest.nlargest(10, 'val_acc')

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
653,100,190,10,90,0.0001,0.317655,0.865789,0.292571,0.894000,0:11:41,1,True
1240,160,140,10,80,0.0010,0.309508,0.870714,0.298903,0.886250,0:12:35,3,True
1178,80,190,30,50,0.0010,0.333464,0.858750,0.292957,0.884583,0:05:13,2,True
1278,200,130,10,90,0.0010,0.302942,0.873192,0.296192,0.882000,0:17:44,3,True
1129,200,200,20,70,0.0001,0.311461,0.867375,0.295427,0.881832,0:29:50,2,True
1107,80,160,10,80,0.0010,0.321133,0.866250,0.336637,0.881250,0:10:42,2,True
1038,100,200,10,90,0.0010,0.306632,0.870800,0.317246,0.881000,0:14:31,2,True
1250,200,190,30,100,0.0010,0.284835,0.881026,0.298489,0.878500,0:28:42,3,True
1267,200,150,20,60,0.0010,0.310335,0.870633,0.298475,0.878250,0:13:40,3,True
1039,100,190,10,50,0.0001,0.344512,0.852526,0.317177,0.878000,0:07:40,2,True


### Analyze Generations

In [24]:
# read generations file
df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')

# check performance of top performing algorithms (dead and alive)
df_generations.sort_values('val_acc', ascending=False).head(30)

,batch_size,steps_per_epoch,validation_steps,epochs,learning_rate,train_loss,train_acc,val_loss,val_acc,time,gen,alive
653,100,190,10,90,0.0001,0.317655,0.865789,0.292571,0.894000,0:11:41,1,True
1240,160,140,10,80,0.0010,0.309508,0.870714,0.298903,0.886250,0:12:35,3,True
1178,80,190,30,50,0.0010,0.333464,0.858750,0.292957,0.884583,0:05:13,2,True
1278,200,130,10,90,0.0010,0.302942,0.873192,0.296192,0.882000,0:17:44,3,True
1129,200,200,20,70,0.0001,0.311461,0.867375,0.295427,0.881832,0:29:50,2,True
1107,80,160,10,80,0.0010,0.321133,0.866250,0.336637,0.881250,0:10:42,2,True
1038,100,200,10,90,0.0010,0.306632,0.870800,0.317246,0.881000,0:14:31,2,True
1250,200,190,30,100,0.0010,0.284835,0.881026,0.298489,0.878500,0:28:42,3,True
1267,200,150,20,60,0.0010,0.310335,0.870633,0.298475,0.878250,0:13:40,3,True
1039,100,190,10,50,0.0001,0.344512,0.852526,0.317177,0.878000,0:07:40,2,True


### Total Training Time of All Generations

In [27]:
df_generations = pd.read_csv(os.path.join(path_project, 'generations.csv')).fillna('')

times = df_generations.time.tolist()
times = list(map(lambda x: re.sub(r'^0:', '', x), times))

minutes = sum(list(map(lambda x: int(x.split(':')[0]), times))) + sum(list(map(lambda x: int(x.split(':')[1]), times))) / 60
hours = minutes/60
print(round(hours, 2), 'hours')

237.54 hours
